In [12]:
import os
from diarize import srt_to_transcript
from transcriptTools import datesDict, titlesDict, speakerLabels, getTranscriptFiles

ROOT = os.getcwd()

labels = speakerLabels("speaker_labels.csv")
rotl_titles = titlesDict("rotl_titles.txt")
rotl_dates = datesDict("rotl_dates.txt")
roadwork_titles = titlesDict("roadwork_titles.txt")
roadwork_dates = datesDict("roadwork_dates.txt")


def getTitle(showname, episode):
    if showname == "rotl":
        return rotl_titles[episode]
    else:
        return roadwork_titles[episode]


def getDate(showname, episode):
    if showname == "rotl":
        return rotl_dates[episode]
    else:
        return roadwork_dates[episode]


def getLabel(showname, episode, speaker):
    return labels[showname][episode][speaker]


def getDuration(start, end):
    if start > end:
        return 0.000
    else:
        return round(end - start, 3)


transcriptDir = os.path.join(ROOT, "transcripts")
files = getTranscriptFiles(transcriptDir)

In [13]:
import sqlite3

dbFile = "transcripts.db"

conn = sqlite3.connect(dbFile)

c = conn.cursor()

c.execute(
    """CREATE TABLE lines (
        id integer primary key,
        filename text,
        showname text,
        episode text,
        title text,
        date text,
        idx integer,
        start real,
        end real,
        duration real,
        speaker text,
        label text,
        speech text
        )"""
)

conn.commit()

In [14]:
for file in files:
    filepath, showname, filename = file
    episode = filename.split("_-_")[0]
    title = getTitle(showname, episode)
    date = getDate(showname, episode)
    transcript = srt_to_transcript(filepath)
    for idx, start, end, speaker, speech in transcript:
        label = getLabel(showname, episode, speaker)
        duration = getDuration(start, end)
        c.execute(
            "INSERT INTO lines VALUES (null, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (
                filename,
                showname,
                episode,
                title,
                date,
                idx,
                start,
                end,
                duration,
                speaker,
                label,
                speech,
            ),
        )
    conn.commit()
conn.close()

In [27]:
searchterm = "bed"
conn = sqlite3.connect("transcripts.db")
c = conn.cursor()

c.execute("""SELECT * FROM lines WHERE speech LIKE '%' || ? || '%'""", (searchterm,))

results = c.fetchall()

conn.commit()
conn.close()

len(results)

3339

In [28]:
lines = [
    speech
    for id, filename, showname, episode, title, date, idx, start, end, duration, speaker, label, speech in results
]

lines

["And I feel like I have, and it occurred to me over the last, because what I do is as I'm on my way up to bed, I will, you know, we have like a window in our door.",
 'My lights... When I put the kids to bed, I come downstairs and I turn on the main lights on the outside of the house.',
 "And it's not like if it doesn't happen that I can't sleep or something like that or like I'll be tossing and turning in bed if I don't do whatever the thing is.",
 'is there a certain time a day when you would wake up or is it entirely based on when you went to bed the night before?',
 "So, I mean, our typical listener is probably going to be... Well, either as I've just described.",
 "But when the car dealer, when you're testing out of bed and, It's a sham because you don't really get to lay on the bed the way you would really lay on a bed if it was in your house.",
 "You're laying on a bed.",
 "And you're laying on the bed in a public way as if you're sort of pretending.",
 'This, everyone in the s